In [1]:
import pandas as pd
import glob
import numpy as np
import functools
import sys
import pickle
from datetime import date

In [2]:
with open('cleanedMTAData','rb') as read_file:
    df_filter_months = pickle.load(read_file)

In [3]:
df_filter_months.dtypes

C/A                      object
UNIT                     object
SCP                      object
STATION                  object
LINENAME                 object
DIVISION                 object
DATE                     object
TIME                     object
DESC                     object
ENTRIES                 float64
EXITS                   float64
TIMESTAMP        datetime64[ns]
EXITS_DIFF              float64
ENTRIES_DIFF            float64
DAY_OF_WEEK              object
TOTAL_TRAFFIC           float64
HOURTIME                  int64
TIMEFRAME                object
WEEKFRAME                object
dtype: object

In [4]:
df_station = df_filter_months.groupby(['STATION','TIMESTAMP','DAY_OF_WEEK','DATE'],sort = True).sum().reset_index()

In [5]:
df_station['HOURTIME'] = df_station.TIMESTAMP.apply(lambda x:x.hour)

In [6]:
df_station

,STATION,TIMESTAMP,DAY_OF_WEEK,DATE,ENTRIES,EXITS,EXITS_DIFF,ENTRIES_DIFF,TOTAL_TRAFFIC,HOURTIME
0,1 AV,2019-01-01 03:00:00,Tuesday,01/01/2019,3.041094e+09,1.629408e+09,2121.0,1613.0,3734.0,3
1,1 AV,2019-01-01 07:00:00,Tuesday,01/01/2019,3.041095e+09,1.629408e+09,755.0,431.0,1186.0,7
2,1 AV,2019-01-01 11:00:00,Tuesday,01/01/2019,3.041096e+09,1.629409e+09,1075.0,845.0,1920.0,11
3,1 AV,2019-01-01 15:00:00,Tuesday,01/01/2019,3.041098e+09,1.629412e+09,2306.0,2310.0,4616.0,15
4,1 AV,2019-01-01 19:00:00,Tuesday,01/01/2019,3.041101e+09,1.629415e+09,3555.0,2889.0,6444.0,19
...,...,...,...,...,...,...,...,...,...,...
774949,ZEREGA AV,2019-06-30 05:00:00,Sunday,06/30/2019,1.946279e+06,9.836130e+05,61.0,29.0,90.0,5
774950,ZEREGA AV,2019-06-30 09:00:00,Sunday,06/30/2019,1.946448e+06,9.837040e+05,91.0,169.0,260.0,9
774951,ZEREGA AV,2019-06-30 13:00:00,Sunday,06/30/2019,1.946709e+06,9.838580e+05,154.0,261.0,415.0,13
774952,ZEREGA AV,2019-06-30 17:00:00,Sunday,06/30/2019,1.946954e+06,9.840460e+05,188.0,245.0,433.0,17


In [7]:
labels = ['Morning','Afternoon']
#df_station['TIMEFRAME'] = df_station.cut(df_station['HOURTIME'] , bins = np.arange(0, df['HOURTIME'], 14), labels = labels) 
df_station['TIMEFRAME']=df_station.HOURTIME.apply(lambda x: 'EVENING' if (x >13) else 'MORNING' )

In [8]:
df_station['WEEKFRAME']=df_station.DAY_OF_WEEK.apply(lambda x: 'WEEKEND' if ((x == 'Saturday') or (x == 'Sunday')) else 'WEEK' )

In [9]:
print(df_station)

          STATION           TIMESTAMP DAY_OF_WEEK        DATE       ENTRIES  \
0            1 AV 2019-01-01 03:00:00     Tuesday  01/01/2019  3.041094e+09   
1            1 AV 2019-01-01 07:00:00     Tuesday  01/01/2019  3.041095e+09   
2            1 AV 2019-01-01 11:00:00     Tuesday  01/01/2019  3.041096e+09   
3            1 AV 2019-01-01 15:00:00     Tuesday  01/01/2019  3.041098e+09   
4            1 AV 2019-01-01 19:00:00     Tuesday  01/01/2019  3.041101e+09   
...           ...                 ...         ...         ...           ...   
774949  ZEREGA AV 2019-06-30 05:00:00      Sunday  06/30/2019  1.946279e+06   
774950  ZEREGA AV 2019-06-30 09:00:00      Sunday  06/30/2019  1.946448e+06   
774951  ZEREGA AV 2019-06-30 13:00:00      Sunday  06/30/2019  1.946709e+06   
774952  ZEREGA AV 2019-06-30 17:00:00      Sunday  06/30/2019  1.946954e+06   
774953  ZEREGA AV 2019-06-30 21:00:00      Sunday  06/30/2019  1.947066e+06   

               EXITS  EXITS_DIFF  ENTRIES_DIFF  TOT

In [10]:
df_perStation = df_filter_months.groupby(['STATION'], sort=True).sum().reset_index()
df_perStation = df_perStation.sort_values(by = ['TOTAL_TRAFFIC'],ascending=[False])
print(df_perStation[:10])

             STATION       ENTRIES         EXITS  EXITS_DIFF  ENTRIES_DIFF  \
61    34 ST-PENN STA  4.154431e+12  3.220035e+12  23575040.0    25900944.0   
233  GRD CNTRL-42 ST  1.659207e+12  1.337530e+12  19917500.0    22488284.0   
59   34 ST-HERALD SQ  2.352916e+12  2.121523e+12  16936020.0    18042702.0   
14    14 ST-UNION SQ  8.452163e+11  3.799649e+11  14770557.0    16147319.0   
46             23 ST  4.326012e+12  4.383601e+12  12699933.0    17434097.0   
353   TIMES SQ-42 ST  4.465694e+12  3.487162e+12  14511154.0    15518169.0   
226        FULTON ST  4.818997e+12  4.869018e+12  12273939.0    14449763.0   
68   42 ST-PORT AUTH  3.953878e+12  2.782216e+12  12152566.0    14374171.0   
110            86 ST  5.622906e+11  4.428001e+11  11714135.0    12635808.0   
9             125 ST  3.571598e+12  2.359602e+12   9921647.0    11920333.0   

     TOTAL_TRAFFIC  HOURTIME  
61      49475984.0   1224497  
233     42405784.0    779701  
59      34978722.0    647034  
14      30917876.

In [11]:
with open('top10station_janjun2019', 'wb') as to_write:
    pickle.dump(df_perStation, to_write)